# Initialisation: Revision Record

In [6]:
import sys
from pathlib import Path

# Find the nearest directory that has a "src" folder (works no matter where you run the notebook)
here = Path.cwd()
src_dir = next(p for p in [here, *here.parents] if (p / "src").exists()) / "src"
sys.path.insert(0, str(src_dir))

In [8]:
from ReIDF import df_CLS, eg_CLS
updatedIDF='ref/HVACDetailed-Jul.idf'
df_1= df_CLS(updatedIDF, 'BuildingSurface:Detailed')

In [9]:
print(df_1['Field 2'].unique())
filtered_df = df_1[df_1['Field 2'] == 'Floor,']
filtered_df

['Wall,' 'Floor,' 'Roof,' 'Ceiling,']


,Field 2,Field 3,Field 4,Field 5,Field 6,Field 7,Field 8,Field 9,Field 10,Field 11,Field 12,Field 13,Field 14,Field 15,Field 16,Field 17,Field 18,Field 19,Field 20
Unit Name,,,,,,,,,,,,,,,,,,,
"Zone1-1_Atrium_122d7807..Face12,","Floor,","Generic Interior Floor,","Zone1-1_Atrium_122d7807,","Zone1-1_Atrium_122d7807_Space,","Surface,","Zone4-1Circulation_11_7cbae69e..Face5,","NoSun,","NoWind,",",",",","-44.0569970271814, 24.2703513199043, 15,","-42.1226120889948, 19.9895562671712, 15,","-55.6551969281412, 16.2950068848916, 15,","-56.877013994993, 20.7703410607977, 15;",N/A,N/A,N/A,N/A,N/A
"Zone1-1_Atrium_122d7807..Face15,","Floor,","Generic Interior Floor,","Zone1-1_Atrium_122d7807,","Zone1-1_Atrium_122d7807_Space,","Surface,","Zone4-1Circulation_8_9174b363..Face4,","NoSun,","NoWind,",",",",","-45.1358617417967, 26.6578795939938, 18.5,","-44.0569970271814, 24.2703513199043, 18.5,","-56.877013994993, 20.7703410607977, 18.5,","-57.5584581559348, 23.2663696604741, 18.5;",N/A,N/A,N/A,N/A,N/A
"Zone1-1_Atrium_122d7807..Face17,","Floor,","Generic Interior Floor,","Zone1-1_Atrium_122d7807,","Zone1-1_Atrium_122d7807_Space,","Surface,","Zone4-1Circulation_7_1151c2cf..Face9,","NoSun,","NoWind,",",",",","-52.9982953484024, 6.56317160085695, 22,","-52.0785986477236, 3.19445938715264, 22,","-53.716167084941, 2.74738460210045, 22,","-54.6358637856198, 6.1160968158047, 22;",N/A,N/A,N/A,N/A,N/A
"Zone1-1_Atrium_122d7807..Face18,","Floor,","Generic Interior Floor,","Zone1-1_Atrium_122d7807,","Zone1-1_Atrium_122d7807_Space,","Surface,","Zone4-1Circulation_6_3b628755..Face4,","NoSun,","NoWind,",",",",","-37.9161964662916, 10.6807559183181, 22,","-35.2669635500698, 4.81800185245397, 22,","-51.3249582957785, 0.433986878700496, 22,","-52.9982953484024, 6.56317160085695, 22;",N/A,N/A,N/A,N/A,N/A
"Zone1-1_Atrium_122d7807..Face29,","Floor,","Generic Interior Floor,","Zone1-1_Atrium_122d7807,","Zone1-1_Atrium_122d7807_Space,","Surface,","Zone3-2_OpenOffice2_5_3c05eaa9..Face5,","NoSun,","NoWind,",",",",","-39.8997003674671, 20.8741545301233, 22,","-33.3879173290922, 5.33100281550026, 22,","-35.2669635500698, 4.81800185245397, 22,","-42.0743336822775, 19.8827161235423, 22;",N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Zone8_VRF_4_f441c7bd..Face0,","Floor,","Generic Interior Floor,","Zone8_VRF_4_f441c7bd,","Zone8_VRF_4_f441c7bd_Space,","Surface,","Zone8_VRF_5_1101935f..Face0,","NoSun,","NoWind,",",",",","-67.0082781618199, 5.94939992782831, 8,","-66.1924181854113, 2.96102618986481, 8.0000000...","-68.797478455138, 2.2498151444717, 8,","-69.6133384315467, 5.2381888824352, 8;",N/A,N/A,N/A,N/A,N/A
"Zone8_VRF_5_1101935f..Face5,","Floor,","Ext_G,","Zone8_VRF_5_1101935f,","Zone8_VRF_5_1101935f_Space,","Outdoors,",",","SunExposed,","WindExposed,",",",",","-67.0082781618199, 5.94939992782831, 4.5,","-66.1924181854113, 2.96102618986481, 4.5000000...","-68.797478455138, 2.2498151444717, 4.5,","-69.6133384315467, 5.2381888824352, 4.5;",N/A,N/A,N/A,N/A,N/A
"Zone8_VRF_6_e5e72533..Face9,","Floor,","Ext_G,","Zone8_VRF_6_e5e72533,","Zone8_VRF_6_e5e72533_Space,","Ground,",",","NoSun,","NoWind,",",",",","-64.5662829114424, 13.862179187421, 2.66453525...","-63.5753863578528, 10.2326725031724, 2.6645352...","-66.5560860155406, 9.41890763785151, 3.5527136...","-67.5469825691303, 13.0484143221001, 3.5527136...",N/A,N/A,N/A,N/A,N/A


In [10]:
import re
import numpy as np
import pandas as pd

# --- Step 1: extract z-values including scientific notation ---
def extract_z(value):
    # Capture last number (float or scientific) before ';'
    match = re.search(r'([-+]?\d*\.?\d+(?:[eE][-+]?\d+)?)(?=;\s*$)', str(value).strip())
    return float(match.group()) if match else np.nan

filtered_df['z_value'] = filtered_df['Field 15'].apply(extract_z)

# --- Step 2: Treat very small scientific values as zero ---
# If the absolute value is extremely small (< 1e-6), set to 0
filtered_df.loc[filtered_df['z_value'].abs() < 1e-6, 'z_value'] = 0

# --- Step 3: Get unique sorted z-values ---
unique_z_sorted = np.sort(filtered_df['z_value'].dropna().unique())

print(unique_z_sorted)


[-2.99  0.    4.5   8.   11.5  15.   18.5  22.  ]


C:\Users\s2589602\AppData\Local\Temp\ipykernel_10992\874901899.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['z_value'] = filtered_df['Field 15'].apply(extract_z)


In [18]:
import re
import numpy as np
import pandas as pd

# -----------------------------
# Helpers
# -----------------------------
num = r'[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?'  # float or scientific

def is_na_like(v) -> bool:
    if v is None:
        return True
    s = str(v).strip().lower()
    return s in {"", "na", "n/a", "nan", "none"}

def pick_last_coord_field(row, field_names):
    """Return the LAST non-NA field that ends with ';'. If none, last non-NA."""
    for col in reversed(field_names):
        val = row.get(col)
        if not is_na_like(val):
            s = str(val).strip()
            if s.endswith(';'):
                return s
    for col in reversed(field_names):
        val = row.get(col)
        if not is_na_like(val):
            return str(val).strip()
    return None

def extract_z_from_triplet(text):
    """Extract the last numeric before trailing ';' (supports scientific)."""
    if not text:
        return np.nan
    m = re.search(fr'({num})(?=;\s*$)', str(text))
    return float(m.group(1)) if m else np.nan

# Canonical z→floor mapping you gave
z_to_floor = {
    -2.99: -1, 0.00: 0, 4.50: 1, 8.00: 2,
    11.50: 3, 15.00: 4, 18.50: 5, 22.00: 6
}

def map_floor_from_z(z, mapping, tol=0.05):
    """
    Map z to a floor:
    - try exact (after rounding to 2dp),
    - else snap to nearest canonical z if within tol (default 0.05),
    - else return <NA>.
    """
    if pd.isna(z):
        return pd.NA
    zr = round(float(z), 2)
    if zr in mapping:
        return mapping[zr]
    nearest_key = min(mapping, key=lambda k: abs(zr - k))
    return mapping[nearest_key] if abs(zr - nearest_key) <= tol else pd.NA

# -----------------------------
# 1) Floor-only rows
# -----------------------------
df = df_1.copy()
df['Field 2'] = df['Field 2'].astype(str).str.strip().str.replace(',', '', regex=False)
filtered_df = df[df['Field 2'].eq('Floor')].copy()

# -----------------------------
# 2) Pull last coord from Field 15..19 and extract z
# -----------------------------
coord_fields = [f'Field {i}' for i in range(15, 20)]

filtered_df['coord_text'] = filtered_df.apply(
    lambda r: pick_last_coord_field(r, coord_fields), axis=1
)
filtered_df['z_value'] = filtered_df['coord_text'].apply(extract_z_from_triplet)

# Clamp tiny values to 0 and round to stabilize mapping
filtered_df.loc[filtered_df['z_value'].abs() < 1e-6, 'z_value'] = 0.0
filtered_df['z_value'] = filtered_df['z_value'].round(2)

# Clean zone name
filtered_df['zone'] = filtered_df['Field 4'].astype(str).str.strip().str.rstrip(',')

# Map each row to a floor estimate (nullable int)
filtered_df['floor_est'] = filtered_df['z_value'].apply(lambda z: map_floor_from_z(z, z_to_floor)).astype('Int64')

# -----------------------------
# 3) Mark zones with multiple floors
# -----------------------------
# floors per zone (only known floors; ignore NA)
floors_per_zone = (
    filtered_df.dropna(subset=['floor_est'])
               .groupby('zone')['floor_est']
               .agg(lambda s: sorted(pd.unique(s.astype(int))))
               .rename('floors_found')
)

# counts and flags
zone_counts = floors_per_zone.apply(len).rename('n_floors')
multi_flag = (zone_counts > 1).rename('multi_floor')

# representative z per zone (median), then primary floor
z_by_zone = (
    filtered_df.groupby('zone', as_index=False)['z_value']
               .median()
               .rename(columns={'z_value': 'z_median'})
)

z_by_zone['floor_primary'] = z_by_zone['z_median'].apply(lambda z: map_floor_from_z(z, z_to_floor)).astype('Int64')

# combine
summary = (
    z_by_zone.set_index('zone')
             .join(floors_per_zone, how='left')
             .join(zone_counts, how='left')
             .join(multi_flag, how='left')
             .reset_index()
)

# fill zones that had only NA floors with empty list / zeros
summary['floors_found'] = summary['floors_found'].apply(lambda v: v if isinstance(v, list) else [])
summary['n_floors'] = summary['n_floors'].fillna(0).astype(int)
summary['multi_floor'] = summary['multi_floor'].fillna(False).astype(bool)

# rank zones by primary floor then z
summary = summary.sort_values(['floor_primary', 'z_median'], ascending=[True, True])
summary['level_rank'] = np.arange(1, len(summary) + 1).astype(int)

# Optional: stringify floors_found for a tidy CSV column
summary['floors_found_str'] = summary['floors_found'].apply(lambda lst: '[' + ', '.join(map(str, lst)) + ']')

# -----------------------------
# 4) Save CSVs (summary + detailed)
# -----------------------------
summary_cols = ['zone', 'z_median', 'floor_primary', 'floors_found_str', 'n_floors', 'multi_floor', 'level_rank']
summary.to_csv('ref/zones_by_floor_summary.csv', index=False, columns=summary_cols)

print("Saved: zones_by_floor_summary.csv")

Saved: zones_by_floor_summary.csv


# Floor Areas

In [19]:
import pandas as pd
import numpy as np
import ast

# --- 1) Load data ---
summary = pd.read_csv("ref/zones_by_floor_summary.csv")   # expects columns incl. 'zone', 'floor_primary', 'floors_found_str'
areas   = pd.read_csv("ref/zone_areas.csv")               # expects columns: 'ZONE' (UPPER), 'Area [m2]'

# --- 2) Normalize/prepare keys for joining ---
def norm_zone_key(s: str) -> str:
    if pd.isna(s):
        return ""
    return str(s).strip().rstrip(",").upper()

summary['zone_key'] = summary['zone'].apply(norm_zone_key)
areas['ZONE_KEY']   = areas['ZONE'].apply(norm_zone_key)

# Parse floors_found_str back to list if present; handle missing gracefully
if 'floors_found_str' in summary.columns:
    def parse_list(x):
        if pd.isna(x) or str(x).strip() == "":
            return []
        try:
            # floors_found_str looks like "[0, 1, 2]"; ast.literal_eval handles it
            return ast.literal_eval(str(x))
        except Exception:
            return []
    summary['floors_found'] = summary['floors_found_str'].apply(parse_list)
else:
    summary['floors_found'] = [[] for _ in range(len(summary))]

# --- 3) Join areas to zones (left join from summary) ---
zones = summary.merge(
    areas[['ZONE_KEY', 'Area [m2]']],
    left_on='zone_key',
    right_on='ZONE_KEY',
    how='left'
)

# Optional: see unmatched zones
unmatched = zones[zones['Area [m2]'].isna()][['zone', 'zone_key']]
if not unmatched.empty:
    print(f"Warning: {len(unmatched)} zones had no area match in zone_areas.csv (showing first 10):")
    print(unmatched.head(10).to_string(index=False))

# --- 4) FLOOR AREAS using PRIMARY floor ---
# Use the primary floor for attribution (ignores zones where floor_primary is NA)
primary = zones.dropna(subset=['floor_primary', 'Area [m2]']).copy()
primary['floor_primary'] = primary['floor_primary'].astype('Int64')

floor_areas_primary = (
    primary.groupby('floor_primary', dropna=True, as_index=False)['Area [m2]']
           .sum()
           .rename(columns={'Area [m2]': 'Floor Area [m2]'})
           .sort_values('floor_primary')
)

# --- 5) Alternative: split a multi-floor zone's area equally across its floors ---
#   If a zone has floors_found=[0,1], each floor gets half the area; single-floor zones unaffected.
split_rows = []
for _, r in zones.iterrows():
    area = r.get('Area [m2]')
    floors = r.get('floors_found', [])
    if pd.isna(area) or area is None:
        continue
    if isinstance(floors, float) and np.isnan(floors):
        floors = []
    if not floors:
        # fall back to primary floor if floors_found is empty but primary exists
        fp = r.get('floor_primary')
        if pd.isna(fp):
            continue
        floors = [int(fp)]
    share = area / len(floors)
    for f in floors:
        split_rows.append({'floor': int(f), 'area_share': share})

if split_rows:
    split_df = pd.DataFrame(split_rows)
    floor_areas_split = (
        split_df.groupby('floor', as_index=False)['area_share']
                .sum()
                .rename(columns={'floor': 'floor', 'area_share': 'Floor Area [m2]'})
                .sort_values('floor')
    )
else:
    floor_areas_split = pd.DataFrame(columns=['floor', 'Floor Area [m2]'])

# --- 6) Save outputs ---
zones_out_cols = ['zone', 'floor_primary', 'floors_found_str', 'Area [m2]']

floor_areas_primary.to_csv('ref/floor_areas_primary.csv', index=False)

print("Saved: floor_areas_primary.csv")

Saved: zones_with_areas.csv
Saved: floor_areas_primary.csv


In [ ]:
import pandas as pd

total_area = floor_areas_primary["Floor Area [m2]"].sum()

print(floor_areas_primary.to_string(index=False))
print(f"\nTotal Building Area: {total_area:.2f} m²")

 floor_primary  Floor Area [m2]
            -1           910.88
             0          1680.58
             1          1368.60
             2          1520.63
             3          1520.63
             4          1185.17
             5          1151.32
             6           261.97

Total Building Area: 9599.78 m²
